# Workflow for Phase Retrieval at Various Noise and Number of Pixels

* Prior to the application of phase retrieval algorithms, the data must be pre-processed.

* This workflow demonstrates an example of how raw object- and Fourier-domain images can be processed so that they have different linear number of pixels and different levels of noise subtracted in object and Fourier domains.

* Applies phase retrieval algorithms to these processed images

In [1]:
!pip install opencv-python==3.3.0.10

In [2]:
import sys,os

#datapath = '/Users/Pavel/Documents/temp/on pc/phase_retrieval/BEC/20190124_S#16_20180122_IR792_080mM_800nm_original_AWESOME/20190124_rsp04_ksp16_050fs/35'
#sourcepath = '/Users/Pavel/Documents/repos/PhaseRetrieval/'
#datapath = '/scratch/work/kliuiep1/phase_retrieval/BEC/20190124_S%2316_20180122_IR792_080mM_800nm_original_AWESOME/20190124_rsp04_ksp16_050fs/35/'
#sourcepath = '/scratch/work/kliuiep1/code/'
datapath = '/Users/Pavel/Documents/repos/PhaseRetrieval/docs/data'
sourcepath = '/Users/Pavel/Documents/repos/PhaseRetrieval/'

sys.path.insert(0,sourcepath)
sys.path.insert(0,datapath)

#input filenames
rs_image_raw = 'RS.csv'
ks_image_raw = 'KS.csv'

#automatically track changes in the source code
%load_ext autoreload
%autoreload 2

## Perform image processing and generate object and Fourier domain images with different linear number of pixels and different levels of subtracted noise

In [3]:
os.chdir(sourcepath)
from PhaseRetrieval.classes.rspaceimage import RSpaceImage
from PhaseRetrieval.classes.kspaceimage import KSpaceImage

#generate images with different linear number of pixels
for npixels_pad in [1110, 1500]:
    #
    #subtract different noise levels in k-space 
    for ks_noise in [1900, 2000]:
        #
        #subtract noise in r-space
        for rs_noise in [2500]:
            #
            os.chdir(datapath)
            #
            #read data, rotate and flip
            rs = RSpaceImage(filename=os.path.join(datapath, rs_image_raw),
                             delimiter='\t')
            rs.rotate_image(estimate_only=False, plot_progress = False)
            rs.flip_image(estimate_only=False, plot_progress = False)
            ks = KSpaceImage(filename=os.path.join(datapath, ks_image_raw),
                                         delimiter='\t')
            ks.rotate_image(estimate_only=False, plot_progress = False)
            ks.flip_image(estimate_only=False, plot_progress = False)
            #
            #subtract noise in object domain
            rs.subtract_background(counts=rs_noise, estimate_only=False, log_scale = True, plot_progress = False)
            #
            #apply watershed segmentation in object domain and centre the image
            rs.centre_image_watershed(linear_object_size=100,  
                            npixels_pad=npixels_pad, 
                            apodization=True,
                            plot_progress = False)
            #
            #resample in object domain to make sure the sizes of the pixels are congruent with the digital Fourier transform
            npixels_final = rs.resample_image(fieldofview = 17, 
                             npixels_kspace = 500, 
                             pixelsize_dr0 = 340e-9,
                             lambd = 880e-9, 
                             estimate_only = False)
            #
            #centre image in Fourier domain
            ks.centre_image(estimate_only = False, 
                            centre = (467, 500), 
                            gaussian_filter = True, 
                            sigma = 1,
                            roi=(400,400,550,550), 
                            min_distance = 5, 
                            threshold_abs = 5000, 
                            num_peaks = 5,  
                            zoom = 3, 
                            npixels_pad = npixels_final,
                            plot_progress = False)
            #
            #subtract background in Fourier domain
            ks.subtract_background(counts = ks_noise, 
                                   estimate_only=False,
                                   zoom = 2,
                                   log_scale = True,
                                   plot_progress = False)
            #
            #fulfill Parseval's theorem
            energy_rspace = sum(sum(rs.image))
            ks.renormalise_image(energy_rspace)
            #
            #save images together with their metadata
            ks.save_as_tif(pathtosave=datapath,outputfilename="ks_Ntot"+ str(npixels_pad) + "rsnoise" + str(rs_noise) + "ksnoise" + str(ks_noise) +".tif")
            rs.save_as_tif(pathtosave=datapath,outputfilename="rs_Ntot"+ str(npixels_pad) + "rsnoise" + str(rs_noise) + "ksnoise" + str(ks_noise) + ".tif")

Object domain: Input image was read
Object-domain: Input image was rotated
Object-domain: Image was flipped
Fourier domain: Input image was read
Fourier domain: Input image was rotated
Fourier domain: Input image was flipped
Object domain: Background of 2500 counts was subtracted.
Object domain: Input and watershed images were padded to  1110 X 1110 pixels.
Object domain: Image centred. Apodization filter was applied. Linear pixel size is  283.0 nm
Object domain: Input image shape is  1110 X 1110
Object domain: Image was resampled and its current shape is (277, 277)
Object domain: Image was resampled with the downsampling ratio = 4.0 and zero-padded to npixels_final X npixels_final= 1107 X 1107 pixels
Fourier domain: Input image was padded to  1107 X 1107 pixels and centred.
Fourier domain: Background of 1900 counts has been subtracted.
Fourier domain: Energy =  2736866589583.546
Energy in object's domain * total number of pixels:  2736866589583.546
Fourier domain: Image was renormalis

/Users/Pavel/Documents/repos/PhaseRetrieval/PhaseRetrieval/classes/rspaceimage.py:628: UserWarning: /Users/Pavel/Documents/repos/PhaseRetrieval/docs/data/rs_Ntot1110rsnoise2500ksnoise1900.tif is a low contrast image
  io.imsave(os.path.join(pathtosave, outputfilename), imagetosave.astype(np.float32))


Object domain: Input image was read
Object-domain: Input image was rotated
Object-domain: Image was flipped
Fourier domain: Input image was read
Fourier domain: Input image was rotated
Fourier domain: Input image was flipped
Object domain: Background of 2500 counts was subtracted.
Object domain: Input and watershed images were padded to  1110 X 1110 pixels.
Object domain: Image centred. Apodization filter was applied. Linear pixel size is  283.0 nm
Object domain: Input image shape is  1110 X 1110
Object domain: Image was resampled and its current shape is (277, 277)
Object domain: Image was resampled with the downsampling ratio = 4.0 and zero-padded to npixels_final X npixels_final= 1107 X 1107 pixels
Fourier domain: Input image was padded to  1107 X 1107 pixels and centred.
Fourier domain: Background of 2000 counts has been subtracted.
Fourier domain: Energy =  2736866589583.546
Energy in object's domain * total number of pixels:  2736866589583.546
Fourier domain: Image was renormalis

/Users/Pavel/Documents/repos/PhaseRetrieval/PhaseRetrieval/classes/rspaceimage.py:628: UserWarning: /Users/Pavel/Documents/repos/PhaseRetrieval/docs/data/rs_Ntot1110rsnoise2500ksnoise2000.tif is a low contrast image
  io.imsave(os.path.join(pathtosave, outputfilename), imagetosave.astype(np.float32))


Object domain: Input image was read
Object-domain: Input image was rotated
Object-domain: Image was flipped
Fourier domain: Input image was read
Fourier domain: Input image was rotated
Fourier domain: Input image was flipped
Object domain: Background of 2500 counts was subtracted.
Object domain: Input and watershed images were padded to  1500 X 1500 pixels.
Object domain: Image centred. Apodization filter was applied. Linear pixel size is  283.0 nm
Object domain: Input image shape is  1500 X 1500
Object domain: Image was resampled and its current shape is (500, 500)
Object domain: Image was resampled with the downsampling ratio = 3.0 and zero-padded to npixels_final X npixels_final= 1474 X 1474 pixels
Fourier domain: Input image was padded to  1474 X 1474 pixels and centred.
Fourier domain: Background of 1900 counts has been subtracted.
Fourier domain: Energy =  8660167978687.8
Energy in object's domain * total number of pixels:  8660167978687.795
Fourier domain: Image was renormalised

/Users/Pavel/Documents/repos/PhaseRetrieval/PhaseRetrieval/classes/rspaceimage.py:628: UserWarning: /Users/Pavel/Documents/repos/PhaseRetrieval/docs/data/rs_Ntot1500rsnoise2500ksnoise1900.tif is a low contrast image
  io.imsave(os.path.join(pathtosave, outputfilename), imagetosave.astype(np.float32))


Object domain: Input image was read
Object-domain: Input image was rotated
Object-domain: Image was flipped
Fourier domain: Input image was read
Fourier domain: Input image was rotated
Fourier domain: Input image was flipped
Object domain: Background of 2500 counts was subtracted.
Object domain: Input and watershed images were padded to  1500 X 1500 pixels.
Object domain: Image centred. Apodization filter was applied. Linear pixel size is  283.0 nm
Object domain: Input image shape is  1500 X 1500
Object domain: Image was resampled and its current shape is (500, 500)
Object domain: Image was resampled with the downsampling ratio = 3.0 and zero-padded to npixels_final X npixels_final= 1474 X 1474 pixels
Fourier domain: Input image was padded to  1474 X 1474 pixels and centred.
Fourier domain: Background of 2000 counts has been subtracted.
Fourier domain: Energy =  8660167978687.796
Energy in object's domain * total number of pixels:  8660167978687.795
Fourier domain: Image was renormalis

/Users/Pavel/Documents/repos/PhaseRetrieval/PhaseRetrieval/classes/rspaceimage.py:628: UserWarning: /Users/Pavel/Documents/repos/PhaseRetrieval/docs/data/rs_Ntot1500rsnoise2500ksnoise2000.tif is a low contrast image
  io.imsave(os.path.join(pathtosave, outputfilename), imagetosave.astype(np.float32))


## Apply phase retrieval algorithms

In [4]:
import sys,os
import shutil, glob

#import re
#def sorted_alphanumeric(data):
#    convert = lambda text: int(text) if text.isdigit() else text.lower()
#    alphanum_key = lambda key: [ convert(c) for c in re.split('([0-9]+)', key) ] 
#    return sorted(data, key=alphanum_key)

#set number of phase retrieval reconstruction cycles
rec_number = 3

#number of iterations of Gerchberg-Saxton algorithm
gs_steps = 2

#do not plot progress
plot_progress = False

#final filename to be appended to reconstructed data
filename = 'rs35_ks35'

os.chdir(sourcepath)

#import class for phase retrieval
from PhaseRetrieval.classes.phaseretrieval import PhaseRetrieval


#put files into separate folders
os.chdir(datapath)

#read file names
filenames_list=sorted(glob.glob(os.path.join(datapath, '*.tif')), key=os.path.getmtime)
#object-domain filenames
rs_filenames = [file for file in filenames_list if 'rs_' in file]
#Fourier-domain filenames
ks_filenames = [file for file in filenames_list if 'ks_' in file]

#first go through all object-domain files 
for rs_idx, rs_file in enumerate(sorted(rs_filenames)):
    #
    #create folders with respective file names
    folderName = rs_file[-34:-4]
    #
    #copy object-domain data
    if not os.path.exists(folderName):
        os.mkdir(folderName)
        shutil.copy(rs_file, folderName)
    else:
        shutil.copy(rs_file, folderName)
    #
    #now go through all Fourier-domain files and copy them to the respective folders
    for ks_file in sorted(ks_filenames):
        ks_file = ks_filenames[rs_idx]
        if os.path.exists(folderName):
            shutil.copy(ks_file, folderName)
        else:
            raise ValueError("Invalid path! Object- and Fourier domain filenames must have the same endings!")
                #initialise phase retrieval class
    #
    #initialise phase retrieval class
    pr = PhaseRetrieval(filename_rspace=rs_file, filename_kspace=ks_file)
    #
    #phase retrieval 
    for ii in range(0,rec_number):
        pr.gerchberg_saxton_extrapolation(gs_steps = gs_steps, plot_progress = plot_progress, plot_every_kth_iteration = 1, zoom=1)##
        pr.save_as_tif(filename = filename, pathtosave = os.path.join(datapath, folderName), Fourier_amplitude = True, Fourier_phase = True, object_amplitude = True, object_phase = True)




file type is tif
object-domain data were read as  /Users/Pavel/Documents/repos/PhaseRetrieval/docs/data/rs_Ntot1110rsnoise2500ksnoise1900.tif

object-domain metadata were read as  /Users/Pavel/Documents/repos/PhaseRetrieval/docs/data/rs_Ntot1110rsnoise2500ksnoise1900.csv

file type is tif
Fourier-domain data were read as  /Users/Pavel/Documents/repos/PhaseRetrieval/docs/data/ks_Ntot1110rsnoise2500ksnoise1900.tif

Iteration 1 
  completed 
Iteration 2 
  completed 
saving data as float32 in  /Users/Pavel/Documents/repos/PhaseRetrieval/docs/data/Ntot1110rsnoise2500ksnoise1900
Fourier amplitude saved as  rs35_ks35_Fourier_amplitude_176080000_20200715_171418.tif
Fourier phase saved as  rs35_ks35_Fourier_phase_176080000_20200715_171418.tif
object amplitude saved as  rs35_ks35_object_amplitude_176080000_20200715_171418.tif
object phase saved as  rs35_ks35_object_phase_176080000_20200715_171418.tif
data saved as  rs35_ks35
Iteration 1 
  completed 
Iteration 2 
  completed 
saving data as flo